In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import json

## Read data from Nasa World View

In [2]:
try:
    ds = Dataset('MOD11_L2.A2024279.0145.061.2024279034513.NRT (1).hdf')
    print(ds.variables)
except OSError:
    print("Unable to open the file with netCDF4")


{'Latitude': <class 'netCDF4._netCDF4.Variable'>
>f4 Latitude(Coarse_swath_lines_5km:MOD_Swath_LST, Coarse_swath_pixels_5km:MOD_Swath_LST)
    long_name: Latitude of every 5 scan lines and 5 pixels
    units: degree
    Number Type: float32
    valid_range: [-90.  90.]
    _FillValue: -999.0
    Cell_Along_Swath_Sampling: 3, 2028, 5
    Cell_Across_Swath_Sampling: 3, 1353, 5
unlimited dimensions: 
current shape = (406, 271)
filling on, 'Longitude': <class 'netCDF4._netCDF4.Variable'>
>f4 Longitude(Coarse_swath_lines_5km:MOD_Swath_LST, Coarse_swath_pixels_5km:MOD_Swath_LST)
    long_name: Longitude of every 5 scan lines and 5 pixels
    units: degree
    Number Type: float32
    valid_range: [-180.  180.]
    _FillValue: -999.0
    Cell_Along_Swath_Sampling: 3, 2028, 5
    Cell_Across_Swath_Sampling: 3, 1353, 5
unlimited dimensions: 
current shape = (406, 271)
filling on, 'LST': <class 'netCDF4._netCDF4.Variable'>
>u2 LST(Along_swath_lines_1km:MOD_Swath_LST, Cross_swath_pixels_1km:MOD_S

## Get temperatures in area of interest

In [3]:
file_path = 'MOD11_L2.A2024279.0145.061.2024279034513.NRT (1).hdf'
dataset = Dataset(file_path, 'r')

# Extract latitude, longitude, and LST
lat = dataset.variables['Latitude'][:]
lon = dataset.variables['Longitude'][:]
lst = dataset.variables['LST'][:]
coordinates = np.column_stack((lat.flatten(), lon.flatten()))

last_three_values = lst[:, -3:]- 273.15 


print(last_three_values)
print(coordinates)

[[14.050000000000011 16.03000000000003 17.890000000000043]
 [14.910000000000025 17.190000000000055 17.77000000000004]
 [16.79000000000002 18.07000000000005 18.07000000000005]
 [18.190000000000055 18.190000000000055 18.130000000000052]
 [18.090000000000032 18.03000000000003 17.630000000000052]
 [17.99000000000001 18.250000000000057 17.33000000000004]
 [18.29000000000002 18.170000000000016 16.29000000000002]
 [18.55000000000001 18.170000000000016 15.970000000000027]
 [17.970000000000027 17.670000000000016 15.79000000000002]
 [17.470000000000027 17.210000000000036 15.250000000000057]
 [18.03000000000003 18.370000000000005 17.27000000000004]
 [18.170000000000016 18.07000000000005 15.950000000000045]
 [18.130000000000052 17.730000000000018 15.270000000000039]
 [18.110000000000014 17.690000000000055 15.650000000000034]
 [17.590000000000032 17.150000000000034 15.150000000000034]
 [17.610000000000014 16.710000000000036 14.330000000000041]
 [17.510000000000048 16.310000000000002 14.670000000000

In [4]:
coordinates = np.column_stack((lat.flatten(), lon.flatten()))

print(last_three_values)


[[14.050000000000011 16.03000000000003 17.890000000000043]
 [14.910000000000025 17.190000000000055 17.77000000000004]
 [16.79000000000002 18.07000000000005 18.07000000000005]
 [18.190000000000055 18.190000000000055 18.130000000000052]
 [18.090000000000032 18.03000000000003 17.630000000000052]
 [17.99000000000001 18.250000000000057 17.33000000000004]
 [18.29000000000002 18.170000000000016 16.29000000000002]
 [18.55000000000001 18.170000000000016 15.970000000000027]
 [17.970000000000027 17.670000000000016 15.79000000000002]
 [17.470000000000027 17.210000000000036 15.250000000000057]
 [18.03000000000003 18.370000000000005 17.27000000000004]
 [18.170000000000016 18.07000000000005 15.950000000000045]
 [18.130000000000052 17.730000000000018 15.270000000000039]
 [18.110000000000014 17.690000000000055 15.650000000000034]
 [17.590000000000032 17.150000000000034 15.150000000000034]
 [17.610000000000014 16.710000000000036 14.330000000000041]
 [17.510000000000048 16.310000000000002 14.670000000000

In [5]:

def process_array(arr):
    arr = np.array(arr, dtype=object)
    
    processed_array = []
    
    for sub_arr in arr:
        
        count_dashes = list(sub_arr).count(0)
        if count_dashes == len(sub_arr) or count_dashes == 2:
            continue
        
        elif count_dashes == 1:
            sub_arr_list = list(sub_arr)
            non_dash_elements = [float(x) for x in sub_arr_list if x != 0]
            
            median_val = np.median(non_dash_elements)
            
            sub_arr_list[sub_arr_list.index(0)] = median_val
            
            processed_array.append(sub_arr_list)
        else:
            processed_array.append(sub_arr)
    
    processed_array = np.array(processed_array, dtype=float)
    
    return processed_array

def median_of_all_numbers(arr):
    processed_arr = process_array(arr)
    flattened_arr = processed_arr.flatten()
    return np.median(flattened_arr)

newarray = process_array(last_three_values)

median_val = median_of_all_numbers(newarray)
print("Median of all numbers:", median_val)


Median of all numbers: 10.550000000000011


In [6]:
datos = {
    "median": median_val
}

json_file_path = 'datos.json'  
with open(json_file_path, 'w') as json_file:
    json.dump(datos, json_file, indent=4)

print(f'Median value saved to {json_file_path}.')

Median value saved to datos.json.


## Check that the temperatures are right

In [7]:
#lst_array = df['LST (K)'].to_numpy()
#lst_array

In [8]:
'''Tpromedio= 25.0
Alert = False

for value in lst_array:
    if value > Tpromedio:
        print(f"Value {value} is greater than Tpromedio.")
        Alert = True
if(Alert):
    print("Something's wrong")
else:
    print("it's all good")'''

'Tpromedio= 25.0\nAlert = False\n\nfor value in lst_array:\n    if value > Tpromedio:\n        print(f"Value {value} is greater than Tpromedio.")\n        Alert = True\nif(Alert):\n    print("Something\'s wrong")\nelse:\n    print("it\'s all good")'